In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [4]:
!pip install -U pyarrow #pyarrow is spark and pandas compatible. No need to do pickling and unpickling

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.7/dist-packages (3.0.0)


In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [9]:
import sys,tempfile, urllib.request  #use urllib.request instead of urllib
from pyspark.sql.functions import *

In [7]:

BASE_DIR = '/tmp'
CHURN_DATA_FILE = os.path.join(BASE_DIR, 'Churn_Modelling.csv')

In [12]:
churn_data=urllib.request.urlretrieve('https://raw.githubusercontent.com/AbhiroopC96/Apache-Spark-on-Google-Colab/main/Churn_Modelling.csv',CHURN_DATA_FILE)

In [13]:
!ls /tmp

blockmgr-bc9d8fae-d933-4e86-84a3-afda2f806823
Churn_Modelling.csv
dap_multiplexer.904bf7d9be55.root.log.INFO.20210304-080512.47
dap_multiplexer.INFO
debugger_1v7geqxsgz
hsperfdata_root
initgoogle_syslog_dir.0
spark-67cf2b41-a228-422a-a652-4a53c1bacb33
spark-7be2add3-a292-46fb-9aea-9d9ec4f5e07c


In [16]:
churn_df = spark.read.option("inferSchema", "true").csv("/tmp/Churn_Modelling.csv", header=True)

In [17]:
churn_df.count()

10000

In [18]:
churn_df.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93

In [22]:
churn_df.filter("Geography='Germany'")

DataFrame[RowNumber: int, CustomerId: int, Surname: string, CreditScore: int, Geography: string, Gender: string, Age: int, Tenure: int, Balance: double, NumOfProducts: int, HasCrCard: int, IsActiveMember: int, EstimatedSalary: double, Exited: int]

In [23]:
churn_df.filter("Geography='Germany'").show()

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        8|  15656148|    Obinna|        376|  Germany|Female| 29|     4|115046.74|            4|        1|             0|      119346.88|     1|
|       16|  15643966|   Goforth|        616|  Germany|  Male| 45|     3|143129.41|            2|        0|             1|       64327.26|     0|
|       17|  15737452|     Romeo|        653|  Germany|  Male| 58|     1|132602.88|            1|        1|             0|        5097.67|     1|
|       27|  15736816|     Young|        756|  Germany|  Male| 36|     2|136815.64|            1|        1|             1|  

In [26]:
churn_df.filter("Geography='Germany' and Surname like 'C%' ").show()  #people from Germany and their surnames starting with a C

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|       40|  15585768|   Cameron|        582|  Germany|  Male| 41|     6| 70349.48|            2|        0|             1|      178074.04|     0|
|       51|  15616550|Chidiebele|        698|  Germany|  Male| 44|    10|116363.37|            2|        1|             0|      198059.16|     0|
|       62|  15773469|     Clark|        687|  Germany|Female| 27|     9|152328.88|            2|        0|             0|      126494.82|     0|
|       68|  15641582|   Chibugo|        735|  Germany|  Male| 43|    10|123180.01|            2|        1|             1|  

In [28]:
churn_gender_filter_df=churn_df.filter(col('Gender').startswith('M'))

In [29]:
churn_gender_filter_df.show()

+---------+----------+-------------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|      Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+-------------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        6|  15574012|          Chu|        645|    Spain|  Male| 44|     8|113755.78|            2|        1|             0|      149756.71|     1|
|        7|  15592531|     Bartlett|        822|   France|  Male| 50|     7|      0.0|            2|        1|             1|        10062.8|     0|
|        9|  15792365|           He|        501|   France|  Male| 44|     4|142051.07|            2|        0|             1|        74940.5|     0|
|       10|  15592389|           H?|        684|   France|  Male| 27|     2|134603.88|            1|      

In [33]:
churn_df.first()

Row(RowNumber=1, CustomerId=15634602, Surname='Hargrave', CreditScore=619, Geography='France', Gender='Female', Age=42, Tenure=2, Balance=0.0, NumOfProducts=1, HasCrCard=1, IsActiveMember=1, EstimatedSalary=101348.88, Exited=1)

In [35]:
churn_df.take(5)

[Row(RowNumber=1, CustomerId=15634602, Surname='Hargrave', CreditScore=619, Geography='France', Gender='Female', Age=42, Tenure=2, Balance=0.0, NumOfProducts=1, HasCrCard=1, IsActiveMember=1, EstimatedSalary=101348.88, Exited=1),
 Row(RowNumber=2, CustomerId=15647311, Surname='Hill', CreditScore=608, Geography='Spain', Gender='Female', Age=41, Tenure=1, Balance=83807.86, NumOfProducts=1, HasCrCard=0, IsActiveMember=1, EstimatedSalary=112542.58, Exited=0),
 Row(RowNumber=3, CustomerId=15619304, Surname='Onio', CreditScore=502, Geography='France', Gender='Female', Age=42, Tenure=8, Balance=159660.8, NumOfProducts=3, HasCrCard=1, IsActiveMember=0, EstimatedSalary=113931.57, Exited=1),
 Row(RowNumber=4, CustomerId=15701354, Surname='Boni', CreditScore=699, Geography='France', Gender='Female', Age=39, Tenure=1, Balance=0.0, NumOfProducts=2, HasCrCard=0, IsActiveMember=0, EstimatedSalary=93826.63, Exited=0),
 Row(RowNumber=5, CustomerId=15737888, Surname='Mitchell', CreditScore=850, Geograph

In [36]:
churn_df.filter("Geography='Germany'").sort(col("Surname"), ascending=False).show()


+---------+----------+----------------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|         Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|     6386|  15706810|          Zuyeva|        606|  Germany|Female| 32|     1|106301.85|            2|        0|             1|       59061.25|     0|
|     6974|  15583394|           Zuyev|        659|  Germany|  Male| 39|     8|106259.63|            2|        1|             1|      198103.32|     0|
|     8709|  15776939|             Zox|        778|  Germany|Female| 48|     3|102290.56|            2|        1|             0|      182691.31|     0|
|     4511|  15657747|            Zito|        611|  Germany|Female| 43|     9|127216.31

In [38]:
churn_df.filter("Geography='Germany'").orderBy(col("Surname"), ascending=True).show()

+---------+----------+-----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|    Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+-----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|     6085|  15619494|    Abdulov|        562|  Germany|Female| 31|     9| 117153.0|            1|        1|             1|      108675.01|     0|
|     5461|  15668894|   Abramova|        661|  Germany|  Male| 41|     5|122552.48|            2|        0|             1|       120646.4|     0|
|     8952|  15636388|     Abrego|        702|  Germany|Female| 23|     7| 98775.23|            1|        1|             0|      114603.96|     0|
|     8807|  15598940|     Achebe|        681|  Germany|  Male| 38|     6|181804.34|            2|        1|          

In [40]:
churn_df.filter("Geography='Germany'").sortWithinPartitions([col("IsActiveMember"),col("NumOfProducts")], ascending=False).show()

+---------+----------+----------------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|         Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|     1877|  15664754|          Steele|        640|  Germany|  Male| 39|     9|131607.28|            4|        0|             1|        6981.43|     1|
|     2197|  15791700|Ugochukwutubelum|        773|  Germany|  Male| 47|     2|118079.47|            4|        1|             1|      143007.49|     1|
|     2286|  15783615|           Ramos|        630|  Germany|  Male| 50|     3|129370.91|            4|        1|             1|       47775.34|     1|
|     2618|  15602735|             Kuo|        692|  Germany|  Male| 45|     6|152296.83

In [41]:

churn_df.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [42]:
churn_df.filter('CreditScore>700').sort(col('Age')).show()

+---------+----------+-----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|    Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+-----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|     2022|  15795519|   Vasiliev|        716|  Germany|Female| 18|     3| 128743.8|            1|        0|             0|      197322.13|     0|
|     3518|  15757821|    Burgess|        771|    Spain|  Male| 18|     1|      0.0|            2|        0|             0|       41542.95|     0|
|     2137|  15621893|   Bellucci|        727|   France|  Male| 18|     4|133550.67|            1|        1|             1|       46941.41|     0|
|     3687|  15665327|   Cattaneo|        706|   France|  Male| 18|     2| 176139.5|            2|        1|          

In [44]:
max_age=churn_df.agg({"Age":"max"})

In [46]:
max_age.show()

+--------+
|max(Age)|
+--------+
|      92|
+--------+



In [47]:
import pyspark.sql.functions as F
churn_df.groupBy("Geography","Age").agg(F.max("NumOfProducts")).show()

+---------+---+------------------+
|Geography|Age|max(NumOfProducts)|
+---------+---+------------------+
|  Germany| 54|                 3|
|   France| 92|                 2|
|    Spain| 77|                 1|
|    Spain| 42|                 3|
|   France| 75|                 2|
|    Spain| 43|                 2|
|    Spain| 74|                 2|
|   France| 80|                 2|
|    Spain| 49|                 3|
|    Spain| 51|                 3|
|  Germany| 74|                 2|
|    Spain| 88|                 2|
|    Spain| 18|                 2|
|    Spain| 23|                 2|
|   France| 47|                 4|
|   France| 25|                 3|
|  Germany| 19|                 2|
|  Germany| 48|                 3|
|   France| 42|                 4|
|    Spain| 61|                 2|
+---------+---+------------------+
only showing top 20 rows



In [49]:
churn_df.groupBy("Geography").agg({'NumOfProducts':'sum'}).show()

+---------+------------------+
|Geography|sum(NumOfProducts)|
+---------+------------------+
|  Germany|              3813|
|   France|              7676|
|    Spain|              3813|
+---------+------------------+



In [50]:
pd=churn_df.toPandas()

In [51]:
pd.corr()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,1.000000,0.004202,0.005840,0.000783,-0.006495,-0.009067,0.007246,0.000599,0.012044,-0.005988,-0.016571
CustomerId,0.004202,1.000000,0.005308,0.009497,-0.014883,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
CreditScore,0.005840,0.005308,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,0.000783,0.009497,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.006495,-0.014883,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,-0.009067,-0.012419,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.007246,0.016972,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,0.000599,-0.014025,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.012044,0.001665,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.005988,0.015271,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097


In [52]:
churn_df.createOrReplaceTempView("churn")

In [53]:
spark.sql("select * from churn").show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93